In [1]:
import tensorflow as tf
from tensorflow import keras
import json
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
dataset_path = "data_final.json"

In [3]:
def load_dataset(dataset_path):
    
    with open(dataset_path, 'r') as fp:
        
        data = json.load(fp)
        
    inputs = np.array(data['mfcc'])
    targets = np.array(data['labels'])
    
    return inputs, targets

In [4]:
def prepare_dataset(test_size, validation_size):
    
    inputs, targets = load_dataset(dataset_path)
    
    x_train, x_test, y_train, y_test = train_test_split(inputs, targets, test_size = test_size)
    
    x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size = validation_size)
    
    x_train = x_train.reshape(-1, 130, 13, 1)
    x_test = x_test.reshape(-1, 130, 13, 1)
    x_validation = x_validation.reshape(-1, 130, 13, 1)
    
    return x_train, x_validation, x_test, y_train, y_validation, y_test

In [5]:
x_train, x_validation, x_test, y_train, y_validation, y_test = prepare_dataset(0.15, 0.15)
x_train.shape

(7221, 130, 13, 1)

In [6]:
model_cnn = keras.Sequential([
    keras.layers.Conv2D(128, (3,3), activation='relu', input_shape = (x_train.shape[1],x_train.shape[2],x_train.shape[3])),
    keras.layers.MaxPool2D((3,3), strides=(2,2), padding='same'),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(128, (3,3), activation='relu'),
    keras.layers.MaxPool2D((3,3), strides=(2,2), padding='same'),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(64, (2,2), activation='relu'),
    keras.layers.MaxPool2D((2,2), strides=(2,2), padding='same'),
    keras.layers.BatchNormalization(),
    
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(256,activation= 'relu'),
    keras.layers.Dropout(0.1),
    
    keras.layers.Dense(64, activation = 'relu'),
    keras.layers.Dropout(0.02),
    
    keras.layers.Dense(10, activation='softmax')
])
    
    
model_cnn.summary()
    
    

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 11, 128)      1280      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 6, 128)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 6, 128)        512       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 62, 4, 128)        147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 2, 128)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 31, 2, 128)        512       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 1, 64)         3

In [ ]:
model_cnn.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

model_cnn.fit(x_train, y_train, validation_data=(x_validation, y_validation), batch_size=32, epochs=100)

Epoch 1/100


In [17]:
model_cnn.evaluate(x_test, y_test)

47/47 [==============================] - 2s 38ms/step - loss: 1.0518 - accuracy: 0.8080


[1.0518426895141602, 0.8080000281333923]